In [13]:
import polars as pl

awardsplayers = pl.read_csv("AwardsPlayers.csv")
fielding = pl.read_csv("Fielding.csv")

awardsplayers = awardsplayers.rename({
    'notes': 'POS'
})

summed_stats = fielding.filter(pl.col('yearID') >= 2013).group_by(['playerID', 'yearID', 'lgID', 'POS']).agg([
    pl.col('G').sum().alias('Tot_G'),
    pl.col('GS').sum().alias('Tot_GS'),
    pl.col('InnOuts').sum().alias('Tot_InnOuts'),
    pl.col('PO').sum().alias('Tot_PO'),
    pl.col('A').sum().alias('Tot_A'),
    pl.col('E').sum().alias('Tot_E'),
    pl.col('DP').sum().alias('Tot_DP'),
    pl.col('PB').sum().alias('Tot_PB'),
    pl.col('WP').sum().alias('Tot_WP'),
    pl.col('SB').sum().alias('Tot_SB'),
    pl.col('CS').sum().alias('Tot_CS'),
    pl.col('ZR').sum().alias('Tot_ZR')
])

joined_df = summed_stats.join(
    awardsplayers, 
    on=["playerID", "yearID", "lgID", "POS"],
    how="left"
)

joined_df = joined_df.with_columns(
    pl.when(pl.col('awardID') == 'Gold Glove')
    .then(pl.lit('Yes'))
    .otherwise(pl.lit('No'))
    .alias('Won Gold Glove')
)

joined_df = joined_df.drop('awardID')

final_df = joined_df.with_columns(
    pl.when(pl.col('yearID') == 2023)
      .then(pl.lit('Validation'))
      .otherwise(pl.lit('Training'))
      .alias('Training-Validation')
)

#final_df = final_df.sort('playerID', 'yearID')
final_df

playerID,yearID,lgID,POS,Tot_G,Tot_GS,Tot_InnOuts,Tot_PO,Tot_A,Tot_E,Tot_DP,Tot_PB,Tot_WP,Tot_SB,Tot_CS,Tot_ZR,tie,Won Gold Glove,Training-Validation
str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str
"""paganem01""",2017,"""AL""","""P""",34,0,151,0,2,0,0,null,null,null,null,null,null,"""No""","""Training"""
"""gosseph01""",2016,"""NL""","""1B""",6,4,123,44,7,0,3,null,null,null,null,null,null,"""No""","""Training"""
"""krausma01""",2015,"""AL""","""1B""",20,16,428,110,11,1,13,null,null,null,null,null,null,"""No""","""Training"""
"""hellije01""",2015,"""NL""","""P""",27,27,438,11,19,2,0,null,null,null,null,null,null,"""No""","""Training"""
"""davieza01""",2017,"""NL""","""P""",33,33,574,12,36,0,2,null,null,null,null,null,null,"""No""","""Training"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""sanchca01""",2022,"""NL""","""SS""",1,0,3,0,1,1,0,null,null,null,null,null,null,"""No""","""Training"""
"""jacksja01""",2021,"""NL""","""P""",23,1,65,0,1,0,0,null,null,null,null,null,null,"""No""","""Training"""
"""valaipa01""",2019,"""NL""","""SS""",7,3,102,4,12,1,3,null,null,null,null,null,null,"""No""","""Training"""
